# Inteligencia Artificial: Evaluación 2

## Hecho por:
* David Daniel Henriquez Leal
* Santiago Andres Mercado Barandica

## Preprocesamiento

Guardar la información del espectrograma y otras características de la canción en un archivo csv "dataset_caracteristicas_canciones.csv". (Archivo anexado en el envio de la actividad para no tener que ejecutar esta celda de codigo que toma aproximadamente 4 minutos en ejecutar, solo es necesario cargarlo en el almacenamiento de la sesión)

Este código procesa archivos MP3 para extraer características musicales y generar un dataset. Utilizando la biblioteca `librosa`, se cargan los archivos de audio y se extraen características clave como los coeficientes cepstrales en frecuencias de Mel (MFCC), el Chroma, los centroides espectrales y el espectrograma de Mel. Estas características se promedian a lo largo del tiempo y las bandas melódicas se normalizan para cada canción. El género musical se identifica a partir del nombre del archivo y se guarda junto con las características extraídas.

Finalmente, todas las características y la información de las canciones se almacenan en un DataFrame, que luego se estandariza usando `StandardScaler`. El resultado se guarda en un archivo CSV que incluye las características musicales, el género y el nombre de cada canción, creando así un dataset listo para su análisis.

Referencias:
- https://librosa.org/doc/latest/index.html
- https://scikit-learn.org/dev/modules/generated/sklearn.preprocessing.StandardScaler.html
- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [1]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
from sklearn.preprocessing import StandardScaler
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler

# Crear una lista para almacenar los vectores de características de cada canción
caracteristicas_canciones = []

def get_files_from_folder(folder_path):
    file_list = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".mp3"):  # Solo procesar archivos mp3
                file_list.append(os.path.join(root, file))
    return file_list

def extract_genre(filename):
    genres = ['Rock', 'Pop', 'Electronic', 'Vallenato', 'Salsa']
    for genre in genres:
        if genre.lower() in filename.lower():
            return genre
    return 'Unknown'

drive.mount('/content/drive', force_remount=True)
folder_path = '/content/drive/MyDrive/Canciones_IA_Parcial_2'
files = get_files_from_folder(folder_path)

genres = []
song_names = []

for file in files:
    y, sr = librosa.load(file)

    # Obtener múltiples características para una mejor representación
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)

    # Promedio de cada característica a lo largo del tiempo
    promedio_mfcc = np.mean(mfcc, axis=1)
    promedio_chroma = np.mean(chroma, axis=1)
    promedio_spectral_centroid = np.mean(spectral_centroid, axis=1)

    # Calcular el espectrograma de Mel
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=32)

    # Convertir a escala logarítmica
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Calcular el promedio de cada banda mel
    promedio_bandas_mel = np.mean(log_mel_spectrogram, axis=1)

    # Normalizar el promedio de las bandas mel
    scaler = MinMaxScaler(feature_range=(-1, 1))
    normalized_promedio_bandas_mel = scaler.fit_transform(promedio_bandas_mel.reshape(-1, 1))

    # Concatenar las características en un solo vector
    caracteristicas = np.hstack([promedio_mfcc, promedio_chroma, promedio_spectral_centroid, normalized_promedio_bandas_mel.flatten()])
    caracteristicas_canciones.append(caracteristicas)

    # Extraer el género del nombre del archivo
    genre = extract_genre(os.path.basename(file))
    genres.append(genre)

    # Guardar el nombre de la canción (nombre del archivo sin la extensión)
    song_name = os.path.basename(file).replace('.mp3', '')
    song_names.append(song_name)

# Convertir la lista de características a un DataFrame
df_caracteristicas = pd.DataFrame(caracteristicas_canciones)

# Estandarizar los datos
scaler = StandardScaler()
df_caracteristicas_scaled = scaler.fit_transform(df_caracteristicas)

# Guardar el DataFrame en un archivo CSV en el directorio actual
df_caracteristicas_scaled_df = pd.DataFrame(df_caracteristicas_scaled)
df_caracteristicas_scaled_df['Genre'] = genres
df_caracteristicas_scaled_df['SongName'] = song_names
output_path = './dataset_caracteristicas_canciones.csv'  # Ruta relativa en la misma carpeta del cuaderno
df_caracteristicas_scaled_df.to_csv(output_path, index=False)

print(f"El dataset ha sido guardado en {output_path}")


Mounted at /content/drive
El dataset ha sido guardado en ./dataset_caracteristicas_canciones.csv


## Clustering

Este código implementa un modelo de clustering utilizando el algoritmo K-Means para agrupar canciones según las caracteristicas numericas obtenidas en el preprocesamiento. Primero, se carga un dataset desde un archivo CSV que contiene varias características de las canciones y se eliminan las columnas que corresponden al género y al nombre de la canción, dejando únicamente las características numéricas. A continuación, se crea un modelo de K-Means especificando que se quieren formar 5 clusters y se ajusta a los datos. El modelo asigna una etiqueta de cluster a cada canción, indicando a qué grupo pertenece según sus características. Finalmente, se crea un nuevo DataFrame que asocia cada canción con su cluster correspondiente y se ordena para visualizar las canciones y sus clusters.

Referencias:
- https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
- https://pandas.pydata.org/docs/

In [2]:
import pandas as pd
from sklearn.cluster import KMeans

# Cargar el dataset desde el archivo CSV (suponiendo que está en la misma carpeta)
dataset_path = './dataset_caracteristicas_canciones.csv'
df_caracteristicas_scaled = pd.read_csv(dataset_path)

# Separar las características y los géneros
caracteristicas = df_caracteristicas_scaled.drop(columns=['Genre', 'SongName'])  # Eliminar la columna de género y nombre de la canción
canciones = df_caracteristicas_scaled['SongName']  # Guardar los nombres de las canciones
generos = df_caracteristicas_scaled['Genre']  # Guardar los géneros de las canciones

# Crear el modelo KMeans
kmeans = KMeans(n_clusters=5, random_state=42)

# Ajustar el modelo a la matriz de promedios
clusters = kmeans.fit_predict(caracteristicas)

# Asignar las etiquetas de cluster a cada cancion
df_canciones = pd.DataFrame({'Cancion': canciones, 'Género': generos, 'Cluster': clusters})

# Mostrar las canciones y sus respectivos clusters
df_canciones = df_canciones.sort_values(by='Cluster')
df_canciones.head(100)


,Cancion,Género,Cluster
44,De Musica Ligera Shrek Rock,Rock,0
41,Kyoran Hey Kids Rigby Rock,Rock,0
42,Its my Life Gru Rock,Rock,0
95,"Los Caminos De La Vida, Los Diablitos Vallenato",Vallenato,0
39,Like a Prayer Princess Bubblegum Rock,Rock,0
...,...,...,...
56,Party All Night Pop,Pop,4
79,Faker the Champion Pop,Pop,4
34,La Pelea del Siglo Vallenato,Vallenato,4
72,Viajes Interdimensionales Rock,Rock,4


### ¿Qué información se puede extraer de ese dataset que pueda ser aplicada en otros aspectos de la vida de la vida cotidiana (por ejemplo: estado emocional, salud, economía, etc.)?

De este dataset de caracteristicas de canciones se puede extraer información de utilidad para algunas áreas como:

- **La Psicologia:** Las canciones suelen reflejar o inducir emociones. Si los clusters agrupan canciones con características similares, podríamos inferir que ciertos clusters corresponden a estados emocionales (como canciones tristes, alegres, relajantes, etc.). Esto podría usarse en terapias musicales para lograr regular estados de ánimo en personas segun sus necesidades.

- **La Salud:** Los patrones musicales pueden impactar en la salud mental y el bienestar de las personas. Los resultados del clustering de canciones puede ayudar a crear playlists personalizadas que favorezcan la relajación, concentración o motivación durante actividades como el estudio, el yoga y el ejercicio.

- **La Economía:** En un contexto económico, entender qué tipo de música prefieren los usuarios a partir de los distintos clusters puede ayudar a las plataformas de streaming o productores musicales a optimizar sus estrategias de mercado, aumentando la venta de música o publicidad dirigida a audiencias específicas para cada cluster.

### ¿Qué técnicas de agrupamiento considera usted que se pueden aplicar a este dataset para entender los datos?

* **K-Means (utilizado en el código):** Es un algoritmo de clustering basado en la partición de datos en "k" clusters que minimizan la variación dentro de cada cluster. K-Means es útil para agrupar canciones que comparten características similares, aunque es importante seleccionar el número correcto de clusters, que en nuestro caso fue 5 por el numero de generos presente en el dataset de canciones.

* **DBSCAN (Density-Based Spatial Clustering):** Este método agrupa puntos basados en la densidad, permitiendo detectar agrupaciones que no forman clusters esféricos. Puede ser útil para descubrir subgrupos naturales en los datos, especialmente si hay ruido o variabilidad en las características de las canciones.

* **Agglomerative Clustering (Agrupamiento Jerárquico):** Genera una jerarquía de clusters, lo que facilita la exploración de relaciones entre las canciones en distintos niveles de agrupación.

* **Gaussian Mixture Models (GMM):** Este algoritmo permite que los puntos pertenezcan a más de un cluster con cierta probabilidad, lo que puede reflejar mejor la realidad de las canciones que tienen características de múltiples géneros o estilos, proporcionando una clasificación más flexible.

### Observaciones de los clusters formados

Los clusters obtenidos muestran que géneros como el rock y la electrónica se agrupan de manera compacta y homogénea, con el rock concentrado principalmente en el cluster 0 y la electrónica predominantemente en el cluster 2 y 4. Esto refleja la consistencia de sus características musicales, donde las progresiones armónicas y estructuras tonales en el rock, y los ritmos repetitivos junto con el uso de sintetizadores en la electrónica, facilitan su agrupación. En contraste, el pop es el género más disperso, pues incorpora una amplia variedad de elementos, desde diferentes ritmos y tempos hasta variaciones en la instrumentación, lo que dificulta su agrupación en un solo cluster. La salsa se agrupa mayormente en los clusters 0 y 1, mientras que el vallenato se concentra en el cluster 1, ambos géneros con patrones rítmicos bien definidos que facilitan su agrupación. Asimismo, se observan outliers que pueden representar canciones con características poco convencionales dentro de su género. A pesar de la dispersión en géneros más diversos, el K-Means ha sido efectivo para identificar patrones consistentes, especialmente en géneros más homogéneos como el rock y la electrónica, lo que sugiere que las características extraídas capturan bien las particularidades de estos estilos.

## Reducción de dimensiones

Este código realiza una reducción de dimensionalidad utilizando t-SNE (t-distributed Stochastic Neighbor Embedding) para visualizar características de canciones en un plano 2D. Primero, se carga un dataset desde un archivo CSV que contiene diversas características de canciones, así como su género. Luego, se eliminan las columnas correspondientes al género y nombre de las canciones para dejar solo las características numéricas. A continuación, se aplica t-SNE para reducir las dimensiones de estas características a dos componentes principales. El resultado es almacenado en un nuevo DataFrame junto con los géneros de las canciones. Finalmente, se utiliza Plotly para graficar estos datos en un plano 2D, donde cada punto representa una canción y se colorea según su género, facilitando la visualización de cómo se agrupan las canciones de acuerdo a sus características extraídas y comparar los resultados con el género real de la canción.

Referencias:
- https://scikit-learn.org/dev/modules/generated/sklearn.manifold.TSNE.html
- https://github.com/plotly/plotly.py
- https://pandas.pydata.org/docs/

In [3]:
import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px

# Cargar el dataset desde el archivo CSV (suponiendo que está en la misma carpeta)
dataset_path = './dataset_caracteristicas_canciones.csv'
df_caracteristicas_scaled = pd.read_csv(dataset_path)

# Separar las características y los géneros
caracteristicas = df_caracteristicas_scaled.drop(columns=['Genre', 'SongName'])  # Eliminar la columna de género y nombre de la canción
genres = df_caracteristicas_scaled['Genre']  # Guardar la columna de géneros

# Aplicar t-SNE para reducir la dimensionalidad a 2 componentes
tsne = TSNE(n_components=2, perplexity=40, max_iter=2000, random_state=42)
tsne_components = tsne.fit_transform(caracteristicas)

# Crear un DataFrame con los componentes t-SNE, los nombres de las canciones y los géneros
df_tsne = pd.DataFrame(data=tsne_components, columns=['Dim1', 'Dim2'])
df_tsne['Genero'] = genres

# Graficar los datos en un plano 2D usando Plotly, con colores según el género
fig = px.scatter(df_tsne, x='Dim1', y='Dim2', color='Genero', hover_data=['Genero'],
                 title='Reducción de Dimensionalidad con t-SNE por Género')
fig.show()


### ¿Cuántas dimensiones (características, “features”, variables, etc.) identifica usted en este dataset?

En el dataset que se utilizó para este ounto se almacenan 58 dimensiones o características por canción. En particular, se han extraído las siguientes características:

* MFCC: Se han utilizado 13 coeficientes de Mel-Frequency Cepstral Coefficients, lo que aporta 13 características.

* Chroma: Se ha extraído la característica Chroma, que normalmente tiene 12 valores correspondientes a los 12 tonos de una escala musical, lo que aporta 12 características.

* Centroid Espectral: Aporta 1 característica (el promedio del Centroid Espectral).

* Mel Spectrogram: Se ha utilizado un espectrograma de Mel con 32 bandas. Después de calcular el promedio en cada banda y normalizar, esto añade 32 características.

Sumando todas estas características se obtienen:

13 (MFCC) + 12 (Chroma) + 1 (Centroid Espectral) + 32 (Mel Spectrogram) = 58 dimensiones

### ¿Cuál cree usted que es la mejor forma de mostrar la información del dataset?

Considero que la mejor forma de mostrar la información es con un gráfico bidimensional. Para esto se utiliza t-SNE, ya que haciendo pruebas es el algortimo de reducción de dimensiones que mejores resultados demostró. Esta técnica preserva las relaciones no lineales entre las características, lo que facilita la agrupación natural de las canciones en función de sus similitudes. Ya que no podemos ver los datos en 58 dimensiones, hay que reducirlas a un número mucho menor que el ser humano sea capaz de comprende. Al reducir las 58 características originales a solo dos dimensiones, el gráfico permite visualizar cómo las canciones que clasificamos en 5 géneros (Rock, Pop, Electronic, Vallenato, Salsa) se organizan, lo que nos brinda una representación intuitiva de la relación que hay entre ellas. La utilización de Plotly añade interactividad, lo que permite explorar los datos de manera más detallada, proporcionando información adicional sobre cada canción representada por un punto al pasar el cursor. Esta visualización no solo facilita el análisis de la efectividad de las características, sino que también permite comunicar los resultados de forma clara y comprensible, destacando la utilidad de t-SNE como una técnica fundamental para la visualización de datos complejos.

### Observaciones de la gráfica

En general, los géneros musicales están bien distribuidos y separados en el espacio de características. Por ejemplo, Rock tiende a estar en un rango donde Dim1 < -5 y Dim2 < -2, mientras que Electronic se encuentra en un rango de Dim1 > -2 y Dim2 > -1. En la gráfica se puede ver que en general se forman agrupaciones entre canciones que consideramos del mismo género, siendo Pop el más disperso pero esto se debe a las características de este género musical que es muy variado. El solapamiento entre géneros, como el observado entre Electronic y Pop, podría deberse a similitudes en algunas características compartidas, como patrones rítmicos o características espectrales que no son exclusivas de un solo género. También se pueden observar outliers, posiblemente debido a características concretas de esa canción que pueden hacerla más parecida a otros géneros en las características analizadas. A pesar de estas dispersiones y solapamientos, la técnica de t-SNE ha sido efectiva en resaltar las diferencias globales entre los géneros, lo que indica que las características extraídas, aunque no perfectas, permiten capturar algunas de las particularidades de las canciones con el mismo género musical